Borrowed heavily from: https://github.com/nvmoyar/autoencoders/blob/master/Convolutional_Autoencoder_MNIST.ipynb 

In [ ]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#import keras


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)

In [ ]:
i = i+1
print("i:", i)
img = mnist.train.images[i]
plt.imshow(img.reshape((28, 28)), cmap='Greys_r')
print(mnist.train.labels[i])

In [ ]:
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

### Encoder
conv1 = tf.layers.conv2d(inputs_, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16
maxpool1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), padding='same')
# Now 14x14x16
conv2 = tf.layers.conv2d(maxpool1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
#maxpool2 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
encoded = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
# Now 7x7x8
#conv3 = tf.layers.conv2d(maxpool2, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
#encoded = tf.layers.max_pooling2d(conv3, (2,2), (2,2), padding='same')
# Now 4x4x8


#Need some kinda of compression step here...



### Decoder
#upsample1 = tf.image.resize_nearest_neighbor(encoded, (7,7))
# Now 7x7x8
#conv4 = tf.layers.conv2d(upsample1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
#upsample2 = tf.image.resize_nearest_neighbor(conv4, (14,14))
upsample2 = tf.image.resize_nearest_neighbor(encoded, (14,14))
# Now 14x14x8
conv5 = tf.layers.conv2d(upsample2, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
upsample3 = tf.image.resize_nearest_neighbor(conv5, (28,28))
# Now 28x28x8
conv6 = tf.layers.conv2d(upsample3, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16

logits = tf.layers.conv2d(conv6, 1, (3,3), padding='same', activation=None)
#Now 28x28x1

decoded = tf.nn.sigmoid(logits, name='decoded')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
sess = tf.Session()


In [ ]:
epochs = 4
batch_size = 200
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    print(mnist.train.num_examples//batch_size, "batches")
    for ii in range(mnist.train.num_examples//batch_size):
        batch = mnist.train.next_batch(batch_size)
        imgs = batch[0].reshape((-1, 28, 28, 1))
        batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_: imgs,
                                                         targets_: imgs})

        if ii%50==0: print("Epoch: {}/{}, batch {}...".format(e+1, epochs, ii), 
                           "Training loss: {:.4f}".format(batch_cost))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))
in_imgs = mnist.test.images[:10]
reconstructed = sess.run(decoded, feed_dict={inputs_: in_imgs.reshape((10, 28, 28, 1))})

for images, row in zip([in_imgs, reconstructed], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.reshape((28, 28)), cmap='Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)


fig.tight_layout(pad=0.1)

In [ ]:
encoded_stuff = sess.run(encoded, feed_dict={inputs_: in_imgs.reshape((10, 28, 28, 1))})
plt.hist(np.log(encoded_stuff[6].flatten() + 0.1), bins=8)



In [ ]:
plt.imshow([[x[3] for x in xx] for xx in encoded_stuff[9]], cmap='Greys_r')

In [ ]:
28*28

In [ ]:
7*7*8

In [ ]:
plt.imshow(in_imgs[1].reshape(28, 28), cmap='Greys_r')

In [ ]:
encoded_stuff[0].flatten

In [ ]:
plt.imshow(
sess.run(decoded, feed_dict = {encoded: encoded_stuff})[1].reshape(28, 28), cmap='Greys_r')

# Now Freeze the Compressed images...
WIP...

In [ ]:
compressed_files = sess.run(encoded, feed_dict = {inputs_: mnist.train.images.reshape(60000,28,28,1)})

In [ ]:
compressed_test = sess.run(encoded, feed_dict = {inputs_: mnist.test.images.reshape(10000,28,28,1)})

In [ ]:
compressed_files.shape

## Now train component 3 (classifier)

In [ ]:
inputs_comp_ = tf.placeholder(tf.float32, (None, 7, 7, 8), name='inputs_comp')
targets_lab_ = tf.placeholder(tf.int32, (None), name='targets_lab')

### Decoder
#upsample1 = tf.image.resize_nearest_neighbor(encoded, (7,7))
# Now 7x7x8
#conv4 = tf.layers.conv2d(upsample1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
#upsample2 = tf.image.resize_nearest_neighbor(conv4, (14,14))
flat1 = tf.layers.flatten(inputs_comp_)
#dense1 = tf.layers.dense(flat1, 128, activation=tf.nn.relu)
# Now 1*128
drop1 = tf.layers.dropout(flat1, rate = 0.5)
dense2 = tf.layers.dense(drop1, 64, activation=tf.nn.relu)
# Now 1*64
drop2 = tf.layers.dropout(dense2, rate = 0.5)
final = tf.layers.dense(drop2, 10, activation=tf.nn.softmax)
# 1*10

one_hot_labs = tf.one_hot(targets_lab_,10)

#loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=one_hot_labs, logits=final)
loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labs, logits=final)
cost1 = tf.reduce_mean(loss1)
opt1 = tf.train.AdamOptimizer(0.001).minimize(cost1)

In [ ]:
epochs = 50
batch_size = 200
sess.run(tf.global_variables_initializer())

for e in range(epochs):
    print(mnist.train.num_examples//batch_size, "batches")
    for ii in range(mnist.train.num_examples//batch_size):
        imgs = compressed_files[ii*batch_size : (ii+1)*batch_size]
        labs = mnist.train.labels[ii*batch_size : (ii+1)*batch_size]
        #print(labs.shape)
        #imgs = batch[0].reshape((-1, 28, 28, 1))
        batch_cost, _ = sess.run([cost1, opt1], feed_dict={inputs_comp_: imgs,
                                                         targets_lab_: labs})

        if ii%50==0: print("Epoch: {}/{}, batch {}...".format(e+1, epochs, ii), 
                           "Training loss: {:.4f}".format(batch_cost))

In [ ]:
train_pred = sess.run(final, feed_dict = {inputs_comp_: compressed_files})

In [ ]:
train_preds_cat = [x.argmax() for x in train_pred]

In [ ]:
plt.hist(train_preds_cat)

## Just looking at frequencies, maybe in sample is good, but test predictions are AWFUL!

In [ ]:
sum(train_preds_cat == mnist.train.labels)/len(train_preds_cat)

# Test accuracy (spoiler: bad):

In [ ]:
test_preds = sess.run(final, feed_dict = {inputs_comp_: compressed_test})
test_preds_cat = [x.argmax() for x in test_preds]
sum(test_preds_cat == mnist.test.labels)/len(test_preds_cat)

In [ ]:
plt.hist(test_preds_cat)

# Let's get an end-to-end classifier working first

Architecture based loosely on this: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [ ]:
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_lab_ = tf.placeholder(tf.int32, (None), name='targets_lab')

### Encoder
conv1 = tf.layers.conv2d(inputs_, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16
#conv2 = tf.layers.conv2d(conv1, 64, (3,3), padding='same', activation=tf.nn.relu)

maxpool1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), padding='same')
# Now 14x14x16
drop1 = tf.layers.dropout(maxpool1, rate = 0.25)

conv2 = tf.layers.conv2d(drop1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
maxpool1 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
# Now 7x7x8

flat1 = tf.layers.flatten(maxpool1)
dense1 = tf.layers.dense(flat1, 128, activation=tf.nn.relu)
# Now 1*128
dense2 = tf.layers.dense(dense1, 64, activation=tf.nn.relu)
# Now 1*64
final = tf.layers.dense(dense2, 10, activation=tf.nn.softmax)
# 1*10

one_hot_labs = tf.one_hot(targets_lab_,10)

#loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=one_hot_labs, logits=final)
loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_labs, logits=final)
cost1 = tf.reduce_mean(loss1)
opt1 = tf.train.AdamOptimizer(0.001).minimize(cost1)

In [ ]:
#Train the thing
epochs = 3
batch_size = 200
sess.run(tf.global_variables_initializer())

for e in range(epochs):
    print(mnist.train.num_examples//batch_size, "batches")
    for ii in range(mnist.train.num_examples//batch_size):
        imgs = mnist.train.images[ii*batch_size : (ii+1)*batch_size].reshape((-1, 28, 28, 1))
        labs = mnist.train.labels[ii*batch_size : (ii+1)*batch_size]
        #print(labs.shape)
        #imgs = batch[0].reshape((-1, 28, 28, 1))
        batch_cost, _ = sess.run([cost1, opt1], feed_dict={inputs_: imgs,
                                                         targets_lab_: labs})

        if ii%50==0: print("Epoch: {}/{}, batch {}...".format(e+1, epochs, ii), 
                           "Training loss: {:.4f}".format(batch_cost))



In [ ]:
# Train accuracy
train_preds = sess.run(final, feed_dict = {inputs_: mnist.train.images.reshape(-1,28,28,1)})
train_preds_cat = [x.argmax() for x in train_preds]
sum(train_preds_cat == mnist.train.labels)/len(train_preds_cat)

In [ ]:
# Test accuracy
test_preds = sess.run(final, feed_dict = {inputs_: mnist.test.images.reshape(-1,28,28,1)})
test_preds_cat = [x.argmax() for x in test_preds]
sum(test_preds_cat == mnist.test.labels)/len(test_preds_cat)

## Can we compress this?? (specialized framework)

In [ ]:
compressed_train_spec = sess.run(maxpool1, feed_dict = {inputs_: mnist.train.images.reshape(-1,28,28,1)})
compressed_test_spec = sess.run(maxpool1, feed_dict = {inputs_: mnist.test.images.reshape(-1,28,28,1)})

In [ ]:
compressed_train_spec.shape

In [ ]:
inputs_comp_ = tf.placeholder(tf.float32, (None, 7, 7, 8), name='inputs_comp')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

### Decoder
#upsample1 = tf.image.resize_nearest_neighbor(encoded, (7,7))
# Now 7x7x8
#conv4 = tf.layers.conv2d(upsample1, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 7x7x8
#upsample2 = tf.image.resize_nearest_neighbor(conv4, (14,14))
upsample2 = tf.image.resize_nearest_neighbor(inputs_comp_, (14,14))
# Now 14x14x8
conv5 = tf.layers.conv2d(upsample2, 8, (3,3), padding='same', activation=tf.nn.relu)
# Now 14x14x8
upsample3 = tf.image.resize_nearest_neighbor(conv5, (28,28))
# Now 28x28x8
conv6 = tf.layers.conv2d(upsample3, 16, (3,3), padding='same', activation=tf.nn.relu)
# Now 28x28x16

logits = tf.layers.conv2d(conv6, 1, (3,3), padding='same', activation=None)
#Now 28x28x1

decoded = tf.nn.sigmoid(logits, name='decoded')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
#Train the thing
epochs = 10
batch_size = 200
sess.run(tf.global_variables_initializer())

for e in range(epochs):
    print(mnist.train.num_examples//batch_size, "batches")
    for ii in range(mnist.train.num_examples//batch_size):
        imgs = compressed_train_spec[ii*batch_size : (ii+1)*batch_size]
        labs = mnist.train.images[ii*batch_size : (ii+1)*batch_size].reshape((-1, 28, 28, 1))
        #print(labs.shape)
        #imgs = batch[0].reshape((-1, 28, 28, 1))
        batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_comp_: imgs,
                                                         targets_: labs})

        if ii%50==0: print("Epoch: {}/{}, batch {}...".format(e+1, epochs, ii), 
                           "Training loss: {:.4f}".format(batch_cost))



In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))
raw_imgs = mnist.test.images[:10]
in_imgs = compressed_test_spec[:10]
reconstructed = sess.run(decoded, feed_dict={inputs_comp_: in_imgs.reshape((10, 7, 7, 8))})

for images, row in zip([raw_imgs, reconstructed], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.reshape((28, 28)), cmap='Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)


fig.tight_layout(pad=0.1)

In [ ]:
sess.close()